https://machinelearning.apple.com/2017/10/01/hey-siri.html

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import os
os.chdir('/content/drive/My Drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
!pip install tensorflow
import librosa
import tensorflow as tf
import keras
from keras.layers import Input,Dense,LSTM,GRU,GlobalMaxPool1D,Bidirectional,Concatenate,Flatten
from keras.models import Model, Sequential, model_from_json
import numpy as np
import os
from sklearn.utils import shuffle
from sklearn import metrics
import pickle
from scipy.io.wavfile import read,write

ImportError: ignored

In [ ]:
import pickle

with open('help_trimmed_train_x.pickle', 'rb') as f:
  train_x = pickle.load(f)
with open('help_trimmed_train_y.pickle', 'rb') as f:
  train_y = pickle.load(f)

with open('help_trimmed_test_x.pickle', 'rb') as f:
  test_x = pickle.load(f)
with open('help_trimmed_test_y.pickle', 'rb') as f:
  test_y = pickle.load(f)

with open('google_trimmed_train_x.pickle', 'rb') as f:
  google_x = pickle.load(f)
with open('google_trimmed_train_y.pickle', 'rb') as f:
  google_y = pickle.load(f)

with open('google_mix_help_train_x.pickle', 'rb') as f:
  google1_x = pickle.load(f)
with open('google_mix_help_train_y.pickle', 'rb') as f:
  google1_y = pickle.load(f)

In [ ]:
randomize1 = np.arange(len(train_y))
randomize2 = np.arange(len(test_y))
randomize3 = np.arange(len(google_y))
randomize4 = np.arange(len(google1_y))

np.random.shuffle(randomize1)
np.random.shuffle(randomize2)
np.random.shuffle(randomize3)
np.random.shuffle(randomize4)

train_x1 = train_x[randomize1]
train_y1 = train_y[randomize1]

test_x1 = test_x[randomize2]
test_y1 = test_y[randomize2]

google_x1 = google_x[randomize3]
google_y1 = google_y[randomize3]

google1_x1 = google1_x[randomize4]
google1_y1 = google1_y[randomize4]

print(train_x1.shape, train_y1.shape, test_x1.shape, test_y1.shape, google_x1.shape, google_y1.shape, google1_x1.shape, google1_y1.shape)

(3606, 126, 40) (3606,) (139, 126, 40) (139,) (24702, 126, 40) (24702,) (4010, 126, 40) (4010,)


In [ ]:
train_x2 = np.concatenate((train_x1, test_x1, google_x1[:20000], google1_x1))
train_y2 = np.concatenate((train_y1, test_y1, google_y1[:20000], google1_y1))

print(train_x2.shape, train_y2.shape)
print(train_y2.sum())

(27755, 126, 40) (27755,)
6520.0


In [ ]:
def input_frames(x):
  x_new = []
  for sample in x:
    temp = []
    for i in range(2):
      temp.append(sample[i*60 + 3: (i+1)*60 + 3])
    x_new.append(np.array(temp))
  return np.transpose(np.array(x_new), (1,0,2,3))

train_X = input_frames(train_x2)
print(train_X.shape)

(2, 27755, 60, 40)


In [ ]:
def model1(input_shape = (60,40)):

  shared_dense1 = Dense(256, activation = 'elu', name = 'shared_dense1')
  shared_dense2 = Dense(256, activation = 'elu', name = 'shared_dense2')
  shared_dense3 = Dense(128, activation = 'elu', name = 'shared_dense3')
  shared_gru1 = Bidirectional(GRU(128, activation = 'elu', return_sequences=True), name = 'shared_gru1')
  shared_gru2 = Bidirectional(GRU(128, activation = 'elu', return_sequences=True), name = 'shared_gru2')
  gru = Bidirectional(GRU(64, activation = 'elu'), name = 'gru')

  inputs = []
  temp = []
  for i in range(2):
      inputs_i = Input(shape = input_shape, name = 'input_'+str(i+1))
      inputs.append(inputs_i)

      x = shared_dense1(inputs_i)
      x = shared_dense2(x)
      x = shared_dense3(x)
      x = shared_gru1(x)
      x = shared_gru2(x)

      # x = Dense((128), activation = 'elu', name = 'd1')(inputs_i)
      # x = Dense((128), activation = 'elu', name = 'd2')(inputs_i)
      # x = Bidirectional(GRU(128, activation = 'elu', return_sequences=True), name = 'g1')(x)

      model_temp = Model(inputs = inputs_i, outputs = x)
      temp.append(model_temp.output)

  temp = Concatenate(name = 'concat')(temp)
  temp = gru(temp)
  outputs = Dense(1, activation = 'sigmoid', name = 'output')(temp)
  return Model(inputs=inputs, outputs=outputs)

In [ ]:
model = model1()
model.summary()

Model: "model_77"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 60, 40)       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 60, 40)       0                                            
__________________________________________________________________________________________________
shared_dense1 (Dense)           (None, 60, 256)      10496       input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
shared_dense2 (Dense)           (None, 60, 256)      65792       shared_dense1[0][0]       

In [ ]:
from keras.optimizers import Adam
from keras.losses import BinaryCrossentropy,SparseCategoricalCrossentropy
from tensorflow.keras.metrics import Precision
model.compile(optimizer=Adam(learning_rate = 0.0001), loss=BinaryCrossentropy(), metrics=['acc', Precision()])

In [ ]:
model.fit(x = list(train_X), y = train_y2, batch_size = 128, epochs = 10, validation_split = 0.1)

Train on 24979 samples, validate on 2776 samples
Epoch 1/10
24979/24979 [==============================] - 223s 9ms/step - loss: 0.1962 - acc: 0.9267 - precision_6: 0.7132 - val_loss: 0.2800 - val_acc: 0.8790 - val_precision_6: 0.8736
Epoch 2/10
24979/24979 [==============================] - 217s 9ms/step - loss: 0.0770 - acc: 0.9699 - precision_6: 0.9045 - val_loss: 0.1124 - val_acc: 0.9589 - val_precision_6: 0.9187
Epoch 3/10
24979/24979 [==============================] - 220s 9ms/step - loss: 0.0532 - acc: 0.9798 - precision_6: 0.9290 - val_loss: 0.0268 - val_acc: 0.9928 - val_precision_6: 0.9364
Epoch 4/10
24979/24979 [==============================] - 219s 9ms/step - loss: 0.0427 - acc: 0.9839 - precision_6: 0.9420 - val_loss: 0.0988 - val_acc: 0.9629 - val_precision_6: 0.9464
Epoch 5/10
24979/24979 [==============================] - 218s 9ms/step - loss: 0.0250 - acc: 0.9909 - precision_6: 0.9508 - val_loss: 0.2417 - val_acc: 0.9045 - val_precision_6: 0.9543
Epoch 6/10
24979/2497

In [ ]:
model_json = model.to_json()
with open("model_a.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model_a.h5")
print("Saved model to disk")

Saved model to disk


In [18]:
!python -m pip install keras
!pip install --upgrade tensorflow
import tensorflow
import keras
print(keras._version_)
from keras.models import load_model
from keras import backend as K

# load json and create model
model = load_model('speaker_ver1_model.h5')
print("Loaded model from disk")

# !pip3 install imageai --upgrade
# !pip uninstall tensorflow
# !pip install tensorflow==1.15.2

Requirement already up-to-date: tensorflow in /usr/local/lib/python3.6/dist-packages (2.3.0)


ImportError: ignored

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import os
os.chdir('/content/drive/My Drive')

Mounted at /content/drive


In [5]:
import tensorflow as tf
import keras

def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    """
    Freezes the state of a session into a pruned computation graph.

    Creates a new computation graph where variable nodes are replaced by
    constants taking their current value in the session. The new graph will be
    pruned so subgraphs that are not necessary to compute the requested
    outputs are removed.
    @param session The TensorFlow session to be frozen.
    @param keep_var_names A list of variable names that should not be frozen,
                          or None to freeze all the variables in the graph.
    @param output_names Names of the relevant graph outputs.
    @param clear_devices Remove the device directives from the graph for better portability.
    @return The frozen graph definition.
    """
    from tensorflow.python.framework.graph_util import convert_variables_to_constants
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.global_variables()]
        # Graph -> GraphDef ProtoBuf
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ""
        frozen_graph = convert_variables_to_constants(session, input_graph_def,
                                                      output_names, freeze_var_names)
        return frozen_graph


frozen_graph = freeze_session(K.get_session(),
                              output_names=[out.op.name for out in model.outputs])

tf.train.write_graph(frozen_graph, "Models", "speaker_vver1_model.pb", as_text=True)

ImportError: ignored